This first cell is just a diagnostic from an older example project on thermopiles. All that matters is that the packages import and the test passes. 

In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
# quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
# cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
# 1 * @u_str("mA") is 1 milliamp
function power_efficiency(power_in, resistance, seebeck, delta_t)
        voltage = seebeck * delta_t
        return uconvert(Unitful.NoUnits, (voltage ^ 2) / (resistance * power_in))
end
P, R, S, ΔT = @variables P, R, S, ΔT
output = (S * ΔT)^2 / (R * P)
print(
Test.@test power_efficiency(P * @u_str("W"), R * @u_str("Ω"), 
                            S * @u_str("V") / @u_str("K"), ΔT * @u_str("K")) - output == 0
)

Test Passed

All values use units composed of kilograms, meters, and seconds.

In [2]:
using PhysicalConstants
import PhysicalConstants.CODATA2018:c_0
using UnitfulAstro:Lsun,Rsun,AU,GMsun
L_sun, R_sun = uconvert(@u_str("W"), 1*Lsun), uconvert(@u_str("m"), 1*Rsun)
x_0, GM_sun = uconvert(@u_str("m"), 1*AU), uconvert(@u_str("m")^3 / @u_str("s")^2, 1*GMsun)
[x*1 for x in [L_sun, R_sun, x_0, GM_sun, c_0]]

5-element Vector{Quantity{Float64}}:
            3.828e26 W
             6.957e8 m
      1.495978707e11 m
 1.3271244e20 m^3 s^-2
   2.99792458e8 m s^-1

In [3]:
v, c, m = @variables v, c, m
γ = 1 / sqrt(1 - v^2 / c^2)
KE = (γ - 1) * m * c^2
KE

m*(c^2)*(1 / sqrt(1 + (-(v^2)) / (c^2)) - 1)

\begin{align}
T = \left(\frac{1}{\sqrt{1-(v^2/c^2)}}-1\right)m_0c^2\\
\left(\frac{T}{m_0c^2}+1 \right)^{-2} = 1-(v^2/c^2)\\
v = c\sqrt{1-\left(\frac{T}{m_0c^2}+1 \right)^{-2} }\\
\frac{dv_\odot}{dt} = \left[m_0c\left(\frac{T_\odot}{m_0c^2}+1\right)^3\sqrt{1-\left(\frac{T_\odot}{m_0c^2}+1\right)^{-2}} \right]^{-1}\\
dv = dv_g + dv_\odot \quad\therefore\quad \int dv = \int \frac{dv_\odot}{dT_\odot}\frac{dT_\odot}{dt}+\frac{dv_g}{dt}dt
\end{align}

In [10]:
gamma(v, c) = 1 / sqrt(1 - v^2 / c^2)
kinetic_energy(v, m, c) = (gamma(v, c) - 1) * m * c^2
print(Test.@test kinetic_energy(v, m, c ) - KE == 0)

Test Passed

The following output should be equal to 50 J.

In [11]:
kinetic_energy(BigFloat(10)* @u_str("m")/@u_str("s"), 1*@u_str("kg"), c_0)

50.00000000000004172437710201073175070569566477939694833199009616528698564728937 kg m^2 s^-2

In [14]:
solar_power(x, area) =  L_sun * area / (4*pi* x^2)
grav_acc(x) = GM_sun / x^2

grav_acc (generic function with 1 method)

In [87]:
v_from_KE(T, m, c) = c*(sqrt(1 - (T / (m * c^2) + 1)^-2))

v_from_KE (generic function with 1 method)

I am not accounting for the redshifting of Sun light that impacts the spacecraft.

In [136]:
function increment(x_old, v_old, time_step_size, area, mass)
  x_new = x_old + v_old * time_step_size
  KE_old = kinetic_energy(v_old, mass, c_0)
  solar_push = solar_power(x_old, area) * time_step_size
  Δv_light = v_from_KE(KE_old + solar_push, mass, c_0) - v_from_KE(KE_old, mass, c_0)
  Δv_grav = v_old - grav_acc(x_old) * time_step_size
  println(Δv_light)  
  println(Δv_grav)
  v_new = v_old + Δv_light + Δv_grav
  return x_new, v_new
end
increment(10^11 * @u_str("m"), c_0*.99, 10^-5 * @u_str("s"), 
          10^3 * @u_str("m")^2, 1 * @u_str("kg"))

1.5497207641601562e-6 m s^-1
2.9679453341928834e8 m

 s^-1


(1.0000000296794533e11 m, 5.935890668392899e8 m s^-1)

In [130]:
total_time_elapsed, time_step_size = 10^7, 10^-5
time_steps = range(0, time_step_size, total_time_elapsed)
positions, velocities, x_old, v_old = [], [], 10^11 * @u_str("m"), 0 * @u_str("m") / @u_str("s")
area, mass = 1 * @u_str("m")^2, 1 * @u_str("kg")
for time_step in time_steps[1:100]
    try 
        x_new, v_new = increment(x_old, v_old, time_step * @u_str("s"), area, mass)
        push!(positions, x_new)
        push!(velocities, v_new)
        x_old, v_old = x_new, v_new
        #print(x_old, v_old)
    catch
        println(time_step)
        println(x_old)
        println(v_old)
        break
    end
end
positions, velocities

7.500000750000074e-11
9.99999999999634e10 m
-5.013734729500264e8 m s^-1


(Any[1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m, 1.0e11 m  …  9.999999999999994e10 m, 9.999999999999988e10 m, 9.999999999999974e10 m, 9.999999999999948e10 m, 9.999999999999895e10 m, 9.999999999999785e10 m, 9.999999999999562e10 m, 9.99999999999911e10 m, 9.999999999998195e10 m, 9.99999999999634e10 m], Any[0.0 m s^-1, -1.3271245327124532e-14 m s^-1, -5.308498130849813e-14 m s^-1, -1.4598369859836986e-13 m s^-1, -3.450523785052378e-13 m s^-1, -7.564609836460982e-13 m s^-1, -1.5925494392549436e-12 m s^-1, -3.2779975957997592e-12 m s^-1, -6.662165154216515e-12 m s^-1, -1.344377151637715e-11 m s^-1  …  -979245.0643555203 m s^-1, -1.9584901287110406e6 m s^-1, -3.916980257422081e6 m s^-1, -7.833960514844162e6 m s^-1, -1.5667921029688325e7 m s^-1, -3.133584205937665e7 m s^-1, -6.26716841187533e7 m s^-1, -1.253433682375066e8 m s^-1, -2.506867364750132e8 m s^-1, -5.013734729500264e8 m s^-1])

In [66]:
v_old, x_old

(5.628556778786839e8 m s^-1, 1.000151971033102e11 m)

In [124]:
for time_step in range(0, number_of_time_steps, number_of_time_steps)[1:100]
    x_new, v_new = increment(x_old, v_old, time_step_size * @u_str("s"), area, mass)
    println(time_step)
end

UndefVarError: UndefVarError: time_step not defined

In [127]:
total_time_elapsed, time_step_size = BigFloat(10)^7, BigFloat(10)^-5
number_of_time_steps = Integer(floor(total_time_elapsed / time_step_size))
positions, velocities, x_old, v_old = [], [], 10^11 * @u_str("m"), 1 * @u_str("m") / @u_str("s")
area, mass = 1 * @u_str("m")^2, 1 * @u_str("kg")
for step in range(0, number_of_time_steps, number_of_time_steps)[1:100]
    x_new, v_new = increment(x_old, v_old, time_step_size * @u_str("s"), area, mass)
    push!(positions, x_new)
    push!(velocities, v_new)
    x_old, v_old = x_new, v_new
end
positions, velocities

DomainError: DomainError with -0.2622453402304586302310399404208266668993783055576474220885996353898072183925797:
NaN result for non-NaN input.